In [1]:
# 1. 安装系统依赖
!sudo apt update && sudo apt install ffmpeg -y

# 2. 升级 pip 并安装最新版核心库
!pip install --upgrade pip
!pip install -U demucs stable-ts torch tqdm torchcodec

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,225 kB]
Get:5 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:6 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packa

In [3]:
# @title 🛠️ 配置中心 (Configuration & Setup)
import os
import torch
from google.colab import drive

# 1. 挂载 Google Drive
if not os.path.exists("/content/drive"):
    print("正在挂载 Google Drive...")
    drive.mount('/content/drive')

# 2. 文件夹路径设置
# @markdown ### 📂 路径设置
SOURCE_FOLDER = "/content/drive/MyDrive/Music Library/Jay Chou - Love Songs 播放列表"  # @param {type:"string"}
OUTPUT_FOLDER = "/content/drive/MyDrive/lyrics" # @param {type:"string"}

# 3. 模型识别参数
# @markdown ---
# @markdown ### 🤖 模型与识别设置
MODEL_SIZE = "large-v3" # @param ["medium", "large-v3"]
LANGUAGE = "auto" # @param ["auto", "zh", "ja", "en", "ko"]
# 自动跳过文件名包含以下关键词的歌曲
SKIP_KEYWORDS = ["伴奏", "Instrumental", "Off Vocal", "Inst", "カラオケ"] # @param {type:"raw"}

# 4. 歌词切分与精修策略
# @markdown ---
# @markdown ### ✂️ 歌词切分与精修参数 (Post-Processing)
MAX_CHARS_PER_LINE = 24 # @param {type:"integer"}
LIMIT_DUR = 3.0 # @param {type:"number"}
MIN_GAP = 0.8 # @param {type:"number"}
INNER_WORD_OFFSET_MS = 150 # @param {type:"integer"}
SYNC_THRESHOLD = 0.8 # @param {type:"number"}

# 5. 运行参数
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"✅ 配置完成。处理结果将存入: {OUTPUT_FOLDER}")

✅ 配置完成。处理结果将存入: /content/drive/MyDrive/lyrics


In [ ]:
# @title 🚀 开启原子化自动化处理 (V9 Original Logic Alignment)
import re
import gc
import json
import torch
import shutil
import subprocess
import traceback
import torchaudio
import torchaudio.transforms as T
from pathlib import Path
from datetime import datetime
from tqdm.auto import tqdm
import stable_whisper

# ==========================================
# 1. 核心工具函数
# ==========================================
def sanitize_filename(name):
    return re.sub(r'[\\/*?:"<>|]', "", name).replace(" ", "_")

def strip_punc(text):
    """移除歌词中的标点符号 - 严格匹配原始需求并修正警告"""
    # 采用字符集写法，避开转义符号
    punctuation = r'[，。！？、；：""''（）【】《》,.?!()\[\]]'
    return re.sub(punctuation, "", text).strip()

def get_unique_output_path(original_path, output_dir):
    p = Path(original_path)
    safe_name = sanitize_filename(p.stem)
    parent_name = sanitize_filename(p.parent.name)
    return os.path.join(output_dir, f"{safe_name}_{parent_name}.json")

# ==========================================
# 2. 初始化常驻模型 (40GB 显存优化)
# ==========================================
print("📦 正在载入模型 (Whisper & VAD)...")

# 加载 VAD 并强制移动到 GPU 解决运行时设备不匹配问题
vad_model, vad_utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad', force_reload=False, trust_repo=True)
(get_speech_timestamps, _, _, _, _) = vad_utils
vad_model = vad_model.to(DEVICE) # 关键修复：模型移至 GPU

# 加载 Whisper
whisper_model = stable_whisper.load_model(MODEL_SIZE, device=DEVICE)

print("✅ 模型已就绪并全部驻留 GPU。")

# ==========================================
# 3. 单文件原子处理逻辑
# ==========================================
def process_single_file(file_path, output_json_path):
    work_dir = "/content/work"
    if os.path.exists(work_dir): shutil.rmtree(work_dir)
    os.makedirs(work_dir)

    in_m4a = "/content/work/raw.m4a"
    in_wav = "/content/work/raw.wav"
    out_dir = "/content/work/out"

    try:
        # --- A. 物理准备与转码 ---
        shutil.copy2(file_path, in_m4a)
        print(f"\n🎵 预处理转码: {Path(file_path).name}")
        # 转码为 PCM WAV 以解决 Demucs 兼容性问题
        subprocess.run(["ffmpeg", "-y", "-i", in_m4a, "-acodec", "pcm_s16le", "-ar", "44100", "-ac", "2", in_wav],
                       capture_output=True, check=True)

        # --- B. Demucs 分离 (HTDemucs) ---
        print(f"🎸 提取纯净人声 (htdemucs)...")
        subprocess.run([
            "demucs", "-n", "htdemucs", "--two-stems", "vocals", "--device", "cuda", "-o", out_dir, in_wav
        ], check=True)

        # 定位人声文件
        vocal_wav_path = None
        for r, d, f in os.walk(out_dir):
            if "vocals.wav" in f:
                vocal_wav_path = os.path.join(r, "vocals.wav")
                break

        if not vocal_wav_path: raise FileNotFoundError("未找到分离后的人声文件")

        # --- C. 加载音频并预处理 ---
        v_tensor, sr = torchaudio.load(vocal_wav_path)
        resampler = T.Resample(sr, 16000)
        v_16k = resampler(v_tensor).mean(0).to(DEVICE) # 保持在 GPU
        v_np = v_16k.cpu().numpy()

        # --- D. 执行转写 ---
        def run_transcribe(uv):
            args = {"vad": uv, "regroup": False, "language": None if LANGUAGE == "auto" else LANGUAGE}
            fn = str(file_path).lower()
            if LANGUAGE == "zh" or any(z in fn for z in ["zh", "cn", "中文"]):
                args["initial_prompt"] = "歌词转写，无标点。"
            elif LANGUAGE == "ja" or any(j in fn for j in ["ja", "jp", "japanese", "日"]):
                args["initial_prompt"] = "歌詞転写、句読点なし。"
            return whisper_model.transcribe(v_np, **args)

        try: result = run_transcribe(True)
        except: result = run_transcribe(False)

        # --- E. 语义重组 (完全复刻原始代码逻辑) ---
        print(f"🧩 执行重组与切分...")
        algo_str = f'cm_sp={MIN_GAP}'
        try:
            result.regroup(algorithms=algo_str)
        except:
            try:
                result.regroup(regroup_strategy=algo_str)
            except:
                result.regroup()

        # 强制拆分长句与碎片合并
        result.split_by_length(max_chars=MAX_CHARS_PER_LINE)
        result.split_by_duration(max_dur=3.0)
        result.merge_by_gap(0.1, max_words=3)

        # --- F. VAD 时间轴精修 (复刻原始代码后处理逻辑) ---
        raw_dict = result.to_dict()
        # 注意：此处音频和模型都在 CUDA
        speech_ts = get_speech_timestamps(v_16k, vad_model, threshold=0.3)
        vad_segs = [{"start": st['start'] / 16000, "end": st['end'] / 16000} for st in speech_ts]

        refined_segments = []
        off_s = INNER_WORD_OFFSET_MS / 1000.0

        for i, seg in enumerate(raw_dict.get('segments', [])):
            txt = strip_punc(seg.get('text', ''))
            if not txt: continue

            s, e = seg['start'], seg['end']

            # 原始代码中的吸附逻辑改进版
            for v in vad_segs:
                if abs(v['start'] - s) <= SYNC_THRESHOLD: s = v['start']
                if v['start'] <= e <= v['end']: e = v['end']

            # 逐字处理与偏移
            words = []
            for idx, w in enumerate(seg.get('words', [])):
                wt = strip_punc(w['word'])
                if not wt: continue
                # 应用偏移
                ws = w['start'] + (off_s if 0 < idx < len(seg['words'])-1 else 0)
                we = w['end'] + (off_s if 0 < idx < len(seg['words'])-1 else 0)
                words.append({
                    "word": wt,
                    "start": round(max(s, min(ws, e)), 3),
                    "end": round(max(s, min(we, e)), 3)
                })

            refined_segments.append({
                "text": txt,
                "start": round(s, 3),
                "end": round(e, 3),
                "words": words
            })

        # --- G. 保存结果 ---
        output_data = {
            "metadata": {
                "file": Path(file_path).name,
                "duration": round(len(v_np)/16000, 2),
                "language": raw_dict.get("language")
            },
            "segments": refined_segments
        }
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)

        return True

    finally:
        if os.path.exists(work_dir): shutil.rmtree(work_dir)

# ==========================================
# 4. 执行循环
# ==========================================
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
error_log = os.path.join(OUTPUT_FOLDER, "error_log.txt")

tasks = [os.path.join(r, f) for r, d, fs in os.walk(SOURCE_FOLDER) for f in fs
         if f.lower().endswith(".m4a") and not any(k.lower() in f.lower() for k in SKIP_KEYWORDS)]

print(f"🎵 待处理清单: {len(tasks)} 首歌曲。")

pbar = tqdm(tasks)
for f_path in pbar:
    pbar.set_description(f"Processing: {Path(f_path).stem[:20]}")
    out_j = get_unique_output_path(f_path, OUTPUT_FOLDER)
    if os.path.exists(out_j): continue

    try:
        process_single_file(f_path, out_j)
    except Exception as e:
        with open(error_log, "a", encoding="utf-8") as l:
            l.write(f"[{datetime.now()}] {f_path}: {str(e)}\n{traceback.format_exc()}\n---\n")
        print(f"❌ 失败: {Path(f_path).name} (查看日志)")
    finally:
        torch.cuda.empty_cache()
        gc.collect()

print(f"✨ 处理结束！")

<>:25: SyntaxWarning: invalid escape sequence '\['
<>:25: SyntaxWarning: invalid escape sequence '\['
/tmp/ipython-input-336787465.py:25: SyntaxWarning: invalid escape sequence '\['
  punctuation = r'[，。！？、；：""''（）【】《》,.?!()\[\]]'


📦 正在载入模型 (Whisper & VAD)...
Downloading: "https://github.com/snakers4/silero-vad/zipball/master" to /root/.cache/torch/hub/master.zip


100%|██████████████████████████████████████| 2.88G/2.88G [00:25<00:00, 120MiB/s]


✅ 模型已就绪并全部驻留 GPU。
🎵 待处理清单: 24 首歌曲。


  0%|          | 0/24 [00:00<?, ?it/s]

...

🧩 执行重组与切分...

🎵 预处理转码: 16. Coral Sea (feat. Lara Veronin).m4a
🎸 提取纯净人声 (htdemucs)...



VAD:   0%|          | 0/256.33 [00:00<?, ?sec/s]
VAD:   2%|▏         | 4.74/256.33 [00:00<00:05, 47.14sec/s]
VAD:   4%|▎         | 9.47/256.33 [00:00<00:05, 46.00sec/s]
VAD:   5%|▌         | 14.08/256.33 [00:00<00:05, 45.70sec/s]
VAD:   7%|▋         | 18.72/256.33 [00:00<00:05, 45.89sec/s]
VAD:   9%|▉         | 23.33/256.33 [00:00<00:05, 43.49sec/s]
VAD:  11%|█         | 28.26/256.33 [00:00<00:05, 45.32sec/s]
VAD:  13%|█▎        | 33.28/256.33 [00:00<00:04, 46.85sec/s]
VAD:  15%|█▌        | 38.78/256.33 [00:00<00:04, 49.30sec/s]
VAD:  17%|█▋        | 44.16/256.33 [00:00<00:04, 50.62sec/s]
VAD:  19%|█▉        | 49.31/256.33 [00:01<00:04, 50.83sec/s]
VAD:  21%|██▏       | 54.62/256.33 [00:01<00:03, 51.51sec/s]
VAD:  23%|██▎       | 59.78/256.33 [00:01<00:03, 51.32sec/s]
VAD:  25%|██▌       | 64.99/256.33 [00:01<00:03, 51.51sec/s]
VAD:  27%|██▋       | 70.18/256.33 [00:01<00:03, 50.42sec/s]
VAD:  29%|██▉       | 75.23/256.33 [00:01<00:03, 47.62sec/s]
VAD:  31%|███▏      | 80.38/256.33 [0

Detected language: chinese



Transcribe:  10%|▉         | 25.0/256.33 [00:04<00:38,  6.08sec/s]
Transcribe:  20%|██        | 51.5/256.33 [00:07<00:27,  7.36sec/s]
Transcribe:  30%|███       | 77.78/256.33 [00:09<00:20,  8.80sec/s]
Transcribe:  42%|████▏     | 107.78/256.33 [00:11<00:14,  9.96sec/s]
Transcribe:  54%|█████▍    | 137.78/256.33 [00:13<00:09, 12.53sec/s]
Transcribe:  65%|██████▌   | 167.78/256.33 [00:15<00:06, 13.34sec/s]
Transcribe:  77%|███████▋  | 196.58/256.33 [00:17<00:04, 12.56sec/s]
Transcribe:  88%|████████▊ | 226.58/256.33 [00:21<00:02, 11.34sec/s]
Transcribe: 100%|██████████| 256.33/256.33 [00:21<00:00, 11.77sec/s]


🧩 执行重组与切分...

🎵 预处理转码: 17. Love you, no matter what.m4a
🎸 提取纯净人声 (htdemucs)...
